In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np

from tqdm import tqdm
from utils import *

In [ ]:
train_set, train_labels, test_set, test_labels = load_and_preprocess_data(path='./data/CIFAR10')

In [ ]:
mu = 1e-3
rho = 1e-4

input_size = 32 * 32 * 3
hidden_size = 2048
output_size = 10

model = NeuralNetwork(input_size, hidden_size, output_size, sigmoid, d_sigmoid, mu, rho)

In [ ]:
def train_network(model: NeuralNetwork, train_set, train_labels, num_epochs, p_dropout=None):
    N_samples = train_set.shape[0]

    # Add accuracy and loss tracking

    for epoch in range(num_epochs):
        
        data_idx = np.random.permutation(N_samples)
        running_epoch_loss = 0
        running_predictions = []
        running_labels = []
        
        pbar = tqdm(total=N_samples)
        for iter, idx in enumerate(data_idx):
            h = train_set[idx]
            gamma = transform_label(train_labels[idx])

            gamma_hat = model.forward(h)
            loss = cross_entropy_loss(gamma_hat, gamma, model.rho, model.weights())
            model.backward(gamma_hat, gamma)

            running_epoch_loss += loss
    §
            running_predictions.append(np.argmax(gamma_hat))
            running_labels.append(train_labels[idx])

            pbar.update(1)
            pbar.set_description(f'Train loss: {loss:.4f}')
        pbar.close()

        epoch_loss = running_epoch_loss / N_samples
        epoch_accuracy = np.mean(np.array(running_predictions) == np.array(running_labels))

        print(f"Epoch: {epoch + 1} | Train Loss {epoch_loss:.4f} | Train Accuracy: {epoch_accuracy * 100:.2f}%")

In [ ]:
train_network(model, train_set[:10000], train_labels[:10000], num_epochs=10)